In [34]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from tensorflow import keras
from tensorflow.keras import models, layers
import cv2
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import pickle
import glob


path = "Images"

myList = os.listdir(path)
no0fClasses = len(myList)


images =[]
classNo =[]


for i in range(no0fClasses):
    myImageList = os.listdir(path + "\\" + str(i))
    for j in myImageList:
        img=cv2.imread(path + "\\" + str(i) + "\\" + j)
        img=cv2.resize(img,(300,300))
        images.append(img)
        classNo.append(i)
    
    
    
images = np.array(images)
classNo = np.array(classNo)


x_train, x_test, y_train, y_test = train_test_split(images,classNo,test_size=0.5,random_state=42)
x_train, x_validation, y_train, y_validation = train_test_split(x_train,y_train,test_size=0.5,random_state=42)



#fig, axes = plt.subplots(3,1,figsize(7,7))
#fig.subplots_adjust(hspace = 0.5)
#sns.countplot(y_train, ax= axes[0])
#axes[0].set_title("y_train")


#sns.countplot(y_test, ax=axes[1])
#axes[1].set_title(y_test)


#sns.countplot(y_validation, ax=axes[2])
#axes[2].set_title(y_validation)


def preProcess(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    
    return img



x_train = np.array(list(map(preProcess, x_train)))
x_test = np.array(list(map(preProcess, x_test)))
x_validation = np.array(list(map(preProcess, x_validation)))

x_train =x_train.reshape(-1,300,300,1)
x_test =x_test.reshape(-1,300,300,1)
x_validation =x_validation.reshape(-1,300,300,1)


dataGen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            rotation_range=10)

dataGen.fit(x_train)


y_train = to_categorical(y_train, no0fClasses)
y_test = to_categorical(y_test, no0fClasses)
y_validation = to_categorical(y_validation, no0fClasses)


model = Sequential()

model.add(Conv2D(input_shape = (300,300,1), filters = 8, kernel_size = (5,5), activation = "relu", padding = "same"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 16, kernel_size = (3,3), activation = "relu", padding = "same"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=256, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(units=no0fClasses, activation = "sigmoid"))

model.compile(loss = "binary_crossentropy", optimizer = ("Adam"), metrics = ["accuracy"])

batch_size = 10

hist=model.fit_generator(dataGen.flow(x_train, y_train, batch_size = batch_size),
                                     validation_data = (x_validation, y_validation),
                                     epochs = 5, steps_per_epoch = x_train.shape[0]//batch_size, shuffle =1)










model_json = model.to_json()
with open("model.json","w") as file:
  file.write(model_json)

model.save_weights("weights.h5")


Epoch 1/5


C:\Users\dogan\AppData\Local\Temp\ipykernel_28600\1087334405.py:112: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(dataGen.flow(x_train, y_train, batch_size = batch_size),


8/8 [==============================] - 3s 304ms/step - loss: 0.1573 - accuracy: 0.8649 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
8/8 [==============================] - 2s 281ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
8/8 [==============================] - 2s 293ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/5
8/8 [==============================] - 2s 290ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
8/8 [==============================] - 2s 281ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [35]:
import pickle
import glob

model_json = model.to_json()
with open("model.json","w") as file:
  file.write(model_json)

model.save_weights("weights.h5")


In [41]:
import cv2
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')





with open("model.json","r") as file:
    model_json = file.read()
    
loaded_model = model_from_json(model_json)

loaded_model.load_weights("weights.h5")

def preProcess(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img / 255

    return img


def face_detector(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((300,300), np.uint8), img
    for(x,y,w,h) in faces:
        x = x - 50
        w = w + 50
        y = y - 50
        h = h + 50
        cv2.rectangle(img, (x,y),(x+w,y+h),(255,0,0),2)
        roi = img[y:y+h, x:x+w]
    try:
        roi = cv2.resize(roi, (300,300), interpolation = cv2.INTER_AREA)
    except:
        return (x,w,y,h), np.zeros((300,300), np.uint8), img
    return (x,w,y,h), roi, img



acc = []

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    rect, face, image = face_detector(frame)
    
    if np.sum([face]) !=0.0:
        roi = face.astype("float")/255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        
        roi = roi.reshape(-1,300,300,1)
        
        predictions = loaded_model.predict(roi)
        probVal = np.max(predictions)
        classIndex = classNo[0]
        
        

        if probVal > 0:
            confidence = probVal * 100
            dis = str(probVal * 100) + "% similar"
            pkq = "class: " + str(classIndex)

        font = cv2.FONT_HERSHEY_DUPLEX
        color = (250, 120, 255)
        name = dis + " " + pkq
        cv2.putText(image, name, (100, 120), font, 1, color, 2)
        
        if confidence > 80:
            font = cv2.FONT_HERSHEY_DUPLEX
            color = (0, 255, 0)
            name = "Device Unlocked"
            stroke = 2
            cv2.putText(image, name, (250, 450), font, 1, color, 2)
            cv2.imshow('Face Cropper', image)

    else:
        font = cv2.FONT_HERSHEY_DUPLEX
        color = (0, 0, 255)
        name = "Device Locked"
        stroke = 2
        cv2.putText(image, name, (250, 450), font, 1, color, 2)
        cv2.imshow('Face Cropper', image)
        
        
        
 
    
    
    if cv2.waitKey(20) & 0XFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()

<>:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\dogan\AppData\Local\Temp\ipykernel_28600\197253528.py:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


1/1 [==============================] - 0s 29ms/step
